Lets test this with against a known example taken from scipy documentation
Found at https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html


In [1]:
import numpy as np
# total collagen (mg/g dry weight of liver)
x = np.array([7.1, 7.1, 7.2, 8.3, 9.4, 10.5, 11.4])
# free proline (μ mole/g dry weight of liver)
y = np.array([2.8, 2.9, 2.8, 2.6, 3.5, 4.6, 5.0])

Scipy implementation


In [2]:
from scipy import stats
res = stats.spearmanr(x, y)
dof = len(x)-2  # len(x) == len(y)
dist = stats.t(df=dof)
def statistic(x):  # explore all possible pairings by permuting `x`
    rs = stats.spearmanr(x, y).statistic  # type: ignore
    transformed = rs * np.sqrt(dof / ((rs+1.0)*(1.0-rs)))
    return transformed
ref = stats.permutation_test((x,), statistic, permutation_type='pairings')
print(f"Statistic: {ref.statistic:.4f}, P value: {ref.pvalue:.4f}")


Statistic: 2.1918, P value: 0.0913


Our Implementation


In [3]:
import underspecification as usp
sprman = usp.spearman_test(x, y)
print(f"Statistic: {sprman['statistic']:.4f}, P value: {sprman['pvalue']:.4f}")

Statistic: 2.1918, P value: 0.0913
